In [2]:
%load_ext autoreload

%autoreload 1

In [70]:
import xarray as xr
from pathlib import Path
import numpy as np
import numexpr as ne
import dask

import time
from analysis_utils import load_data_set, calc_temp_spac_means

%aimport analysis_utils

In [71]:
# job_path = Path("/users/PAS0409/madelinescyphers/palm/current_version/JOBS/20220308T183847")
job_path = Path("/Users/madelinescyphers/projs/palm/data/20220308T183847")
data_path = job_path / "OUTPUT/20220308T183847_3d.nc"
lad_path = job_path / "INPUT/20220308T183847_static"
ds = load_data_set(data_path)
ds_lad = xr.open_dataset(lad_path, chunks="auto")

In [72]:
try:
    ds.chunks
except ValueError:
    ds = ds.unify_chunks()
try:
    ds_lad.chunks
except ValueError:
    ds_lad = ds_lad.unify_chunks()
     

/Users/madelinescyphers/.conda/envs/palm/lib/python3.9/site-packages/xarray/core/dataset.py:6698: PerformanceWarning: Increasing number of chunks by factor of 18
  return unify_chunks(self)[0]


In [73]:
ds_lad.chunks

Frozen({'zlad': (6,), 'y': (432,), 'x': (192,)})

In [74]:
ds.chunks

Frozen({'x': (64, 64, 64), 'y': (144, 144, 144), 'xu': (64, 64, 64), 'yv': (144, 144, 144), 'time': (24, 24, 24), 'zu_3d': (97, 97), 'zw_3d': (97, 97)})

In [89]:
s = ds.isel(x=[1,2,3,4,5], y=[1,2,3,4,5], time=[1,2,3,4,5], zu_3d=[1,2,3,4,5], xu=[1,2,3,4,5], zw_3d=[1,2,3,4,5], yv=[1,2,3,4,5])

In [98]:
((s.u.values**2 + s.v.values**2 +s.w.values**2)**(1/2)).mean()

1.4358897

In [67]:
ds.u

<xarray.DataArray 'u' (time: 72, zu_3d: 194, y: 432, xu: 192)>
dask.array<open_dataset-b1a2d31fd1c134903f89ae5a2909490eu, shape=(72, 194, 432, 192), dtype=float32, chunksize=(24, 97, 144, 64), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * zu_3d    (zu_3d) float64 0.0 1.5 4.5 7.5 10.5 ... 568.5 571.5 574.5 577.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
Attributes:
    units:      m/s
    long_name:  u

In [69]:
%time ((ds.v[...,1:,0:-2] + ds.v[...,0:-2,0:-2])/2).compute()

/Users/madelinescyphers/.conda/envs/palm/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/Users/madelinescyphers/.conda/envs/palm/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self

CPU times: user 24.9 s, sys: 42.6 s, total: 1min 7s
Wall time: 47.2 s


<xarray.DataArray 'v' (time: 72, zu_3d: 194, yv: 429, x: 190)>
array([[[[            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         ...,
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan],
         [            nan,             nan,             nan, ...,
                      nan,             nan,             nan]],

        [[ 9.10635769e-01,  9.10324454e-01,  9.53532100e-01, ...,
           1.01259840e+00,  9.74060833e-01,  9.27985430e-01],
         [ 4.43166941e-01,  3.67338330e-01,  3.59824628e-01, ...,
           3.92852902e-01,  3.56822342e-01,  3.35830152e-01],
         [-1.47885541e-02, -1.09273665e-01, -1.59744218e-01, ...,
          -1.51178211e-01, -1.88755289e-01, -1.79650769e-01],
...
           4.02614450e+00,  4.02614450e+00,  4.02614450e+00],
         [ 4.02584171e+00,  4.02584219e+00,  4.02584219e+00, ...,
           4.02584124e+00,  4.02584124e+00,  4.02584171e+00],
         [ 4.02553272e+00,  4.02553272e+00,  4.02553272e+00, ...,
           4.02553225e+00,  4.02553225e+00,  4.02553225e+00]],

        [[ 4.02430725e+00,  4.02430773e+00,  4.02430773e+00, ...,
           4.02430725e+00,  4.02430725e+00,  4.02430725e+00],
         [ 4.02397060e+00,  4.02397060e+00,  4.02397060e+00, ...,
           4.02397013e+00,  4.02397013e+00,  4.02397013e+00],
         [ 4.02362823e+00,  4.02362823e+00,  4.02362823e+00, ...,
           4.02362776e+00,  4.02362776e+00,  4.02362776e+00],
         ...,
         [ 4.02620935e+00,  4.02620935e+00,  4.02620935e+00, ...,
           4.02620840e+00,  4.02620888e+00,  4.02620888e+00],
         [ 4.02590704e+00,  4.02590704e+00,  4.02590704e+00, ...,
           4.02590656e+00,  4.02590656e+00,  4.02590656e+00],
         [ 4.02559853e+00,  4.02559900e+00,  4.02559900e+00, ...,
           4.02559805e+00,  4.02559805e+00,  4.02559853e+00]]]],
      dtype=float32)
Coordinates:
  * yv       (yv) float64 3.0 6.0 9.0 12.0 ... 1.281e+03 1.284e+03 1.287e+03
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * zu_3d    (zu_3d) float64 0.0 1.5 4.5 7.5 10.5 ... 568.5 571.5 574.5 577.5
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 556.5 559.5 562.5 565.5 568.5

In [ ]:
    vt = (vt(1:end-1,2:end,:,:) + vt(1:end-1,1:end-1,:,:))/2;
    
    ut = ncread(OutputFile,'u',[1 1 1 t],[96 432 194 12]);
    ut(:,:,1,:) = [];
    ut = (ut(2:end,1:end-1,:,:) + ut(1:end-1,1:end-1,:,:))/2;
    
    wt = ncread(OutputFile,'w',[1 1 1 t],[96 432 194 12]);
    wt(:,:,1,:) = 0;
    wt = (wt(1:end-1,1:end-1,2:end,:) + wt(1:end-1,1:end-1,1:end-1,:))/2;

In [8]:
dz = 3

z_scalar  = np.arange(150//dz, 300//dz)

urban_ratio = 0.5
y_domain = np.arange(int(ds.s.y.size * urban_ratio), ds.s.y.size)

In [9]:
# ds_lad.isel(zlad=1).lad.values

In [ ]:
# ds

In [ ]:
# s = time.time()
# sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)
# print(time.time() - s)
# # ubar_total = np.sqrt(np.square(ds.u.values) + np.square(ds.v.values) + np.square(ds.w.values))

In [5]:
# s = time.time()
# sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# # ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)

# ubar_total = np.sqrt(np.square(sub_ds.u.values) + np.square(sub_ds.v.values) + np.square(sub_ds.w.values))
# print(time.time() - s)

23.01164722442627


In [ ]:
# s = time.time()
# sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
# # ubar = (sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2)**(1/2)

# ubar_total = np.sqrt(np.square(sub_ds.u) + np.square(sub_ds.v) + np.square(sub_ds.w))
# # ubar_total = np.sqrt(np.square(sub_ds.u.values) + np.square(sub_ds.v.values) + np.square(sub_ds.w.values))
# print(time.time() - s)

In [53]:
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar,)# x=1, time=0, xu=1)

In [54]:
s = time.time()
sub_ds.u**2
print(time.time() - s)
sub_ds


0.0021800994873046875


<xarray.Dataset>
Dimensions:  (time: 72, zu_3d: 50, zw_3d: 50, x: 192, xu: 192, y: 216, yv: 216, zpc_3d: 6)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * zu_3d    (zu_3d) float64 148.5 151.5 154.5 157.5 ... 286.5 289.5 292.5 295.5
  * zw_3d    (zw_3d) float64 150.0 153.0 156.0 159.0 ... 288.0 291.0 294.0 297.0
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
  * yv       (yv) float64 648.0 651.0 654.0 ... 1.287e+03 1.29e+03 1.293e+03
  * zpc_3d   (zpc_3d) float64 0.0 1.5 4.5 7.5 10.5 13.5
Data variables: (12/18)
    E_UTM    (x) float64 dask.array<chunksize=(64,), meta=np.ndarray>
    N_UTM    (y) float64 dask.array<chunksize=(72,), meta=np.ndarray>
    Eu_UTM   (xu) float64 dask.array<chunksize=(64,), meta=np.ndarray>
    Nu_UTM   (y) float64 dask.array<chunksize=(72,), meta=np.ndarray>
    Ev_UTM   (x) float64 dask.array<chunksize=(64,), meta=np.ndarray>
    Nv_UTM   (yv) float64 dask.array<chunksize=(72,), meta=np.ndarray>
    ...       ...
    crs      int32 ...
    theta    (time, zu_3d, y, x) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
    v        (time, zu_3d, yv, x) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
    w        (time, zw_3d, y, x) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
    u        (time, zu_3d, y, xu) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
    s        (time, zu_3d, y, x) float32 dask.array<chunksize=(24, 47, 72, 64), meta=np.ndarray>
Attributes: (12/27)
    title:           PALM 6.0  Rev: 4359  run: 20220308T183847.00  host: defa...
    Conventions:     CF-1.7
    creation_time:   2022-03-08 18:41:16 -05
    data_content:    3d
    version:         1
    origin_time:     2019-06-21 12:00:00 +00
    ...              ...
    source:          PALM 6.0 Rev: 4359
    references:      
    keywords:        
    licence:         
    comment:         
    VAR_LIST:        ;theta;v;w;u;s;

In [55]:
sub_ds.u

<xarray.DataArray 'u' (time: 72, zu_3d: 50, y: 216, xu: 192)>
dask.array<getitem, shape=(72, 50, 216, 192), dtype=float32, chunksize=(24, 47, 144, 64), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * zu_3d    (zu_3d) float64 148.5 151.5 154.5 157.5 ... 286.5 289.5 292.5 295.5
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
Attributes:
    units:      m/s
    long_name:  u

In [56]:
sub_ds.v.zu_3d

<xarray.DataArray 'zu_3d' (zu_3d: 50)>
array([148.5, 151.5, 154.5, 157.5, 160.5, 163.5, 166.5, 169.5, 172.5, 175.5,
       178.5, 181.5, 184.5, 187.5, 190.5, 193.5, 196.5, 199.5, 202.5, 205.5,
       208.5, 211.5, 214.5, 217.5, 220.5, 223.5, 226.5, 229.5, 232.5, 235.5,
       238.5, 241.5, 244.5, 247.5, 250.5, 253.5, 256.5, 259.5, 262.5, 265.5,
       268.5, 271.5, 274.5, 277.5, 280.5, 283.5, 286.5, 289.5, 292.5, 295.5])
Coordinates:
  * zu_3d    (zu_3d) float64 148.5 151.5 154.5 157.5 ... 286.5 289.5 292.5 295.5
Attributes:
    units:    meters

In [57]:
sub_ds.w.zw_3d

<xarray.DataArray 'zw_3d' (zw_3d: 50)>
array([150., 153., 156., 159., 162., 165., 168., 171., 174., 177., 180., 183.,
       186., 189., 192., 195., 198., 201., 204., 207., 210., 213., 216., 219.,
       222., 225., 228., 231., 234., 237., 240., 243., 246., 249., 252., 255.,
       258., 261., 264., 267., 270., 273., 276., 279., 282., 285., 288., 291.,
       294., 297.])
Coordinates:
  * zw_3d    (zw_3d) float64 150.0 153.0 156.0 159.0 ... 288.0 291.0 294.0 297.0
Attributes:
    units:    meters

In [24]:
temp = sub_ds.u**2 + sub_ds.v**2 + sub_ds.w**2

In [23]:
%time temp.load()

CPU times: user 3.78 s, sys: 10.2 s, total: 14 s
Wall time: 8.65 s


<xarray.DataArray (zu_3d: 50, y: 216, yv: 216, zw_3d: 50)>
array([[[[15.72673  , 15.726302 , 15.726774 , ..., 15.726437 ,
          15.726431 , 15.726424 ],
         [15.627536 , 15.627108 , 15.62758  , ..., 15.627242 ,
          15.627236 , 15.62723  ],
         [15.68143  , 15.681002 , 15.681474 , ..., 15.681136 ,
          15.68113  , 15.681124 ],
         ...,
         [16.54239  , 16.541962 , 16.542435 , ..., 16.542097 ,
          16.54209  , 16.542084 ],
         [16.62572  , 16.62529  , 16.625763 , ..., 16.625425 ,
          16.625418 , 16.625412 ],
         [16.384722 , 16.384293 , 16.384766 , ..., 16.384428 ,
          16.38442  , 16.384415 ]],

        [[15.72759  , 15.726189 , 15.726829 , ..., 15.726372 ,
          15.726365 , 15.726358 ],
         [15.628395 , 15.626994 , 15.627634 , ..., 15.627177 ,
          15.627171 , 15.627164 ],
         [15.682289 , 15.680888 , 15.681528 , ..., 15.681071 ,
          15.681065 , 15.681058 ],
...
         [16.017162 , 16.017141 , 16.013308 , ..., 16.012814 ,
          16.012808 , 16.012802 ],
         [16.015526 , 16.015505 , 16.011671 , ..., 16.011177 ,
          16.011171 , 16.011166 ],
         [16.01389  , 16.013868 , 16.010035 , ..., 16.00954  ,
          16.009535 , 16.00953  ]],

        [[15.990481 , 15.989479 , 15.986883 , ..., 15.986002 ,
          15.985996 , 15.985991 ],
         [15.992192 , 15.99119  , 15.988594 , ..., 15.987713 ,
          15.987707 , 15.987701 ],
         [15.99391  , 15.992908 , 15.990312 , ..., 15.98943  ,
          15.989425 , 15.989419 ],
         ...,
         [16.017294 , 16.01629  , 16.013697 , ..., 16.012815 ,
          16.01281  , 16.012804 ],
         [16.015657 , 16.014654 , 16.01206  , ..., 16.011179 ,
          16.011173 , 16.011168 ],
         [16.01402  , 16.013018 , 16.010424 , ..., 16.009542 ,
          16.009537 , 16.009531 ]]]], dtype=float32)
Coordinates:
    time     timedelta64[ns] 00:00:05.179000
    x        float64 4.5
    xu       float64 3.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 151.5 154.5 157.5 160.5 ... 289.5 292.5 295.5 298.5
  * yv       (yv) float64 648.0 651.0 654.0 ... 1.287e+03 1.29e+03 1.293e+03
    zw_3d_   (zw_3d) float64 153.0 156.0 159.0 162.0 ... 291.0 294.0 297.0 300.0
Dimensions without coordinates: zu_3d, zw_3d

In [26]:
temp.shape

(50, 216, 216, 50)

In [25]:
%time temp.compute()

CPU times: user 3.4 s, sys: 7.94 s, total: 11.3 s
Wall time: 5.7 s


<xarray.DataArray (zu_3d: 50, y: 216, yv: 216, zw_3d: 50)>
array([[[[15.72673  , 15.726302 , 15.726774 , ..., 15.726437 ,
          15.726431 , 15.726424 ],
         [15.627536 , 15.627108 , 15.62758  , ..., 15.627242 ,
          15.627236 , 15.62723  ],
         [15.68143  , 15.681002 , 15.681474 , ..., 15.681136 ,
          15.68113  , 15.681124 ],
         ...,
         [16.54239  , 16.541962 , 16.542435 , ..., 16.542097 ,
          16.54209  , 16.542084 ],
         [16.62572  , 16.62529  , 16.625763 , ..., 16.625425 ,
          16.625418 , 16.625412 ],
         [16.384722 , 16.384293 , 16.384766 , ..., 16.384428 ,
          16.38442  , 16.384415 ]],

        [[15.72759  , 15.726189 , 15.726829 , ..., 15.726372 ,
          15.726365 , 15.726358 ],
         [15.628395 , 15.626994 , 15.627634 , ..., 15.627177 ,
          15.627171 , 15.627164 ],
         [15.682289 , 15.680888 , 15.681528 , ..., 15.681071 ,
          15.681065 , 15.681058 ],
...
         [16.017162 , 16.017141 , 16.013308 , ..., 16.012814 ,
          16.012808 , 16.012802 ],
         [16.015526 , 16.015505 , 16.011671 , ..., 16.011177 ,
          16.011171 , 16.011166 ],
         [16.01389  , 16.013868 , 16.010035 , ..., 16.00954  ,
          16.009535 , 16.00953  ]],

        [[15.990481 , 15.989479 , 15.986883 , ..., 15.986002 ,
          15.985996 , 15.985991 ],
         [15.992192 , 15.99119  , 15.988594 , ..., 15.987713 ,
          15.987707 , 15.987701 ],
         [15.99391  , 15.992908 , 15.990312 , ..., 15.98943  ,
          15.989425 , 15.989419 ],
         ...,
         [16.017294 , 16.01629  , 16.013697 , ..., 16.012815 ,
          16.01281  , 16.012804 ],
         [16.015657 , 16.014654 , 16.01206  , ..., 16.011179 ,
          16.011173 , 16.011168 ],
         [16.01402  , 16.013018 , 16.010424 , ..., 16.009542 ,
          16.009537 , 16.009531 ]]]], dtype=float32)
Coordinates:
    time     timedelta64[ns] 00:00:05.179000
    x        float64 4.5
    xu       float64 3.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 151.5 154.5 157.5 160.5 ... 289.5 292.5 295.5 298.5
  * yv       (yv) float64 648.0 651.0 654.0 ... 1.287e+03 1.29e+03 1.293e+03
    zw_3d_   (zw_3d) float64 153.0 156.0 159.0 162.0 ... 291.0 294.0 297.0 300.0
Dimensions without coordinates: zu_3d, zw_3d

In [1]:
temp

NameError: name 'temp' is not defined

In [ ]:
u2_2 = np.square(sub_ds.u)

In [58]:
%time u2.compute()

CPU times: user 10.3 s, sys: 27 s, total: 37.3 s
Wall time: 6.57 s


<xarray.DataArray 'u' (time: 72, zu_3d: 50, y: 216, xu: 192)>
array([[[[1.20187120e-03, 5.82910317e-04, 5.15852764e-04, ...,
          7.52276286e-09, 1.77709473e-04, 2.25754575e-05],
         [2.26918129e-08, 5.15691005e-04, 2.44591036e-03, ...,
          3.99560231e-04, 2.30569131e-04, 9.48354170e-07],
         [1.81410671e-03, 2.56682839e-03, 5.87040698e-03, ...,
          8.06077034e-04, 3.45382032e-05, 3.92679649e-05],
         ...,
         [5.80045511e-04, 2.13915337e-05, 9.17660000e-05, ...,
          1.92366529e-03, 1.84880849e-03, 4.38583374e-04],
         [7.68424070e-04, 2.60249805e-03, 2.35586049e-04, ...,
          1.08057319e-03, 2.12310508e-04, 9.75933654e-05],
         [2.22572125e-03, 2.58437311e-03, 5.14582149e-04, ...,
          5.32266240e-05, 8.35219225e-07, 4.84862685e-05]],

        [[1.13997507e-04, 3.34387482e-03, 3.35798359e-05, ...,
          9.64320824e-03, 3.21440655e-03, 1.30557048e-04],
         [4.80260613e-04, 3.38399317e-03, 1.92172464e-03, ...,
          6.77351188e-03, 3.63878487e-03, 2.10809149e-03],
         [2.98883038e-04, 4.52923588e-03, 8.46781023e-03, ...,
          2.79667112e-03, 1.22261979e-03, 1.10638421e-03],
...
          1.33965969e-07, 1.17135912e-07, 1.01032178e-07],
         [8.39452241e-08, 6.95892126e-08, 5.63059821e-08, ...,
          1.32427274e-07, 1.15471749e-07, 9.92742244e-08],
         [8.21748500e-08, 6.78160603e-08, 5.45703749e-08, ...,
          1.30878149e-07, 1.13817222e-07, 9.75445218e-08]],

        [[3.85853241e-07, 3.69492341e-07, 3.52311588e-07, ...,
          4.29273825e-07, 4.15809467e-07, 4.01315447e-07],
         [3.78335955e-07, 3.62651690e-07, 3.46150870e-07, ...,
          4.19762387e-07, 4.06955593e-07, 3.93126584e-07],
         [3.70271863e-07, 3.55286772e-07, 3.39489105e-07, ...,
          4.09642809e-07, 3.97512736e-07, 3.84369343e-07],
         ...,
         [7.91709027e-08, 6.60255921e-08, 5.38107834e-08, ...,
          1.23255532e-07, 1.07883110e-07, 9.31550659e-08],
         [7.75497639e-08, 6.44005453e-08, 5.22169756e-08, ...,
          1.21831846e-07, 1.06365896e-07, 9.15708469e-08],
         [7.59698153e-08, 6.28294572e-08, 5.06878379e-08, ...,
          1.20400955e-07, 1.04859083e-07, 9.00132093e-08]]]],
      dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 649.5 652.5 655.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 151.5 154.5 157.5 160.5 ... 289.5 292.5 295.5 298.5
Dimensions without coordinates: zu_3d

In [60]:
%timeit u2_2.compute()

6.73 s ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
u = sub_ds.u#.values
v = sub_ds.v#.values
w = sub_ds.w#.values
ubar_scalar = ne.evaluate("sqrt(u**2 + v**2 + w**2)")
print(time.time() - s)

20.63394570350647


In [39]:
s = time.time()
sub_ds = ds.isel(y=y_domain, yv=y_domain, zu_3d=z_scalar, zw_3d=z_scalar)
u = sub_ds.u#.values
v = sub_ds.v#.values
w = sub_ds.w#.values
ubar_scalar = ne.evaluate("sqrt(u**2 + v**2 + w**2)")
print(time.time() - s)

22.61529040336609


In [5]:
ubar_scalar = ubar_scalar.mean(skipna=True)

<xarray.DataArray 'u' (time: 72, zu_3d: 193, y: 432, xu: 192)>
[1152589824 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * xu       (xu) float64 0.0 3.0 6.0 9.0 12.0 ... 561.0 564.0 567.0 570.0 573.0
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
    zu_3d_   (zu_3d) float64 1.5 4.5 7.5 10.5 13.5 ... 568.5 571.5 574.5 577.5
Dimensions without coordinates: zu_3d
Attributes:
    units:      m/s
    long_name:  u

In [ ]:
scalar_gradient = ds.s.mean(skipna=True)

In [11]:
TempSpacMeanU, TempSpacMeanV, TempSpacMeanW = calc_temp_spac_means(ds)

In [ ]:
uw = (ds.w - TempSpacMeanW) * (ds.u - TempSpacMeanU);
vw = (ds.w - TempSpacMeanW) * (ds.v - TempSpacMeanV);

In [ ]:
np.power(np.nanmean(uw)**2 + np.nanmean(vw)**2, 1/4)

In [13]:
ustar = (squeeze(nanmean(nanmean(uw,1),2).^2 + nanmean(nanmean(vw,1),2).^2)).^0.25;

KeyError: 'values'

In [8]:
ds.w

<xarray.DataArray 'w' (time: 72, zw_3d: 193, y: 432, x: 192)>
[1152589824 values with dtype=float32]
Coordinates:
  * time     (time) timedelta64[ns] 00:00:05.179000 ... 00:06:00.348000
  * x        (x) float64 1.5 4.5 7.5 10.5 13.5 ... 562.5 565.5 568.5 571.5 574.5
  * y        (y) float64 1.5 4.5 7.5 10.5 ... 1.288e+03 1.292e+03 1.294e+03
    zw_3d_   (zw_3d) float64 3.0 6.0 9.0 12.0 15.0 ... 570.0 573.0 576.0 579.0
Dimensions without coordinates: zw_3d
Attributes:
    units:      m/s
    long_name:  w

In [ ]:
ubar = ubar_total

In [23]:
type(ubar_total)

numpy.ndarray

In [7]:
ne.evaluate("exp(2)")

array(7.3890561)

In [21]:
scalar_grad = ds.s.isel(y=y_domain, zu_3d=z_scalar)

In [16]:
ds.s.y.size * urban_ratio

216.0

In [6]:
ds.s.isel(time=60, zu_3d=10).v

array([[0.1304376 , 0.1125954 , 0.09213795, ..., 0.0807151 , 0.14683247,
        0.14305805],
       [0.12404612, 0.1070879 , 0.07839098, ..., 0.08024671, 0.13751815,
        0.1399681 ],
       [0.12678802, 0.12038978, 0.10291763, ..., 0.08745276, 0.13599472,
        0.13938259],
       ...,
       [0.12819828, 0.14347515, 0.18452443, ..., 0.0778394 , 0.13670161,
        0.14400712],
       [0.13251767, 0.12401052, 0.14600988, ..., 0.07967169, 0.15714346,
        0.1510945 ],
       [0.13553588, 0.11933067, 0.11963697, ..., 0.082677  , 0.15785772,
        0.14794137]], dtype=float32)

In [38]:
da = xr.DataArray(np.arange(25).reshape(5, 5), dims=("x", "y"))
da

<xarray.DataArray (x: 5, y: 5)>
array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])
Dimensions without coordinates: x, y

In [37]:
tgt_x = xr.DataArray(np.arange(0, 5), dims="points")
tgt_y = xr.DataArray(np.arange(0, 5), dims="points")
da = da.isel(x=tgt_x, y=tgt_y)
da

ValueError: Dimensions {'y', 'x'} do not exist. Expected one or more of Frozen({'points': 5})

In [39]:
da.values

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [3]:
a = np.arange(25).reshape((5,5))
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])